# Proper Orthogonal Decomposition for Reduction of Order Modeling.

<a id="table"></a>

## Table of contents
- [Introduction](#intro)
- [High Order Model](#HR)
- [Proper Orthogonal Decomposition](#POD)
- [Low Order Model](#LR)
- [Time improvement](#improvement)
- [Error](#error)

In [1]:
from time import process_time
from scipy import linalg
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

# ROM functions
from modules.rom import rom
# Two phase, two layers, reservoir model
from scripts import reservoir_model_01 as model

# Set image folder path
imgpath = ".\\images"

<a id="intro"></a>

## [Introduction](#table)

$$ Ax = b \\ $$
Where $A \in \mathbb{R}^{n \times n} , x \in \mathbb{R}^{n \times 1} $, and  $b \in \mathbb{R}^{n \times 1}.  $



$$\phi A x= \phi b \\$$

<a id="HR"></a>

## [High Resolution Model](#table)

The reservoir model that we are going to use here is very similar to  <a href = http://nbviewer.jupyter.org/github/JaimeLeal/projects/blob/master/Simulator_Two_Phase_Flow.ipynb?flush_cache=true target=_blank>  Example \#2 in the "Two Phase Flow Examples" </a>. The simulation was run fur 300 timesteps equivalent to 118 days. The Fully Implicit Simulation took  approximately **25 seconds** to run.

In [2]:
# Load the reservoir model 
#exec(open(".\\scripts\\reservoir_model_01.py").read())
sch  = model.sch
# Run simulation
t = process_time()
results, well_solution, sch, info = model.FIM.solve(sch, max_iter= 10, tol = 1E-6, ATS = False)
elapsed_time = process_time() - t
print('Time running simulation = {:.3f} seconds.'.format(elapsed_time))

Time running simulation = 24.133 seconds.


Now, we load the simulation results to see what happens

In [3]:
# Singular value decomposition
_,S_po,_ = linalg.svd(results['p_oil'][:,:50])
_,S_sw,_ = linalg.svd(results['sw'][:,:50])

In [4]:
rom.plot_energy(S_po, "{}\\po_energy".format(imgpath))
rom.plot_energy(S_sw, "{}\\sw_energy".format(imgpath))

**Oil pressure basis vectors**
<img src = ".\images\po_energy.png" width="5000" >
**Water saturation basis vectors**
<img src = ".\images\sw_energy.png" height="10" >

<a id = POD></a>

## [Proper Orthogonal Decomposition](#table)

In [5]:
from modules.rom import rom
x = results['p_oil'][:,:50]
y = results['sw'][:,:50]
Ur_po, Sr_po = rom.pod(x, 99.999, 6)
Ur_sw, Sr_sw = rom.pod(y, 99.99, 6)

Plot the 

In [6]:
rom.plot_basis(Ur_po[:100,:], 2, 3, model.grid, "{}\\basis_po_1st".format(imgpath))
rom.plot_basis(Ur_sw[:100,:], 2, 3, model.grid, "{}\\basis_sw_1st".format(imgpath))

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<img src = ".\images\basis_po_1st.png" height="1" >
<img src = ".\images\basis_sw_1st.png" height="1" >

<a id="LR"></a>

## [Low Order Model](#table)

Now, we will apply the proper orthogonal decomposition technique to the reservoir simulator to speed up the simulation. We will modify the Fully Implicit Solver. After generating the linear syustem to be solved we project it to create the low order model and then solve it with the neton raphson model.
The dimensions of the basis is  x by y so the dimensions of the new system will be k by w. 

In [7]:
sch  = rmodel.sch
# Run simulation
t = process_time()
results, well_solution, sch, info = rmodel.FIM.solve(sch, max_iter= 10, tol = 1E-6, ATS = False)
elapsed_time = process_time() - t
print('Time running simulation = {:.3f} seconds.'.format(elapsed_time))#

NameError: name 'rmodel' is not defined

<a id="improvement"></a>

## [Time Improvement](#table)

<a id="error"></a>

## [Error](#table)

In [8]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */